In [3]:
# DBSCAN to cluster the invidual window

import open3d as o3d
import numpy as np
from sklearn.cluster import DBSCAN

# Load the input point cloud
input_file = "..."  # Update with your file name
pcd = o3d.io.read_point_cloud(input_file)

# Convert point cloud to NumPy array
points = np.asarray(pcd.points)

# Prepare the list to store cluster type and central point
cluster_info = []



# Initial parameters
eps_values = [0.1,0.5]  # Example range of eps
min_samples_values = [5,10]  # Example range of min_samples
min_cluster_size = 50  # Set the minimum number of points in a cluster


for eps in eps_values:
    for min_samples in min_samples_values:
        dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        labels = dbscan.fit_predict(points)
        num_clusters = len(set(labels)) - (1 if -1 in labels else 0)  # Exclude noise
        print(f"eps={eps}, min_samples={min_samples}, clusters={num_clusters}")



# Perform DBSCAN clustering
dbscan = DBSCAN(eps=eps, min_samples=min_samples)
labels = dbscan.fit_predict(points)

# Get unique cluster labels (excluding noise, which is labeled as -1)
unique_labels = set(labels)
unique_labels.discard(-1)  # Remove noise label if present

# Iterate over each cluster and save them as individual files
for cluster_id in unique_labels:
    # Extract points belonging to the current cluster
    cluster_points = points[labels == cluster_id]

    # Skip clusters smaller than the minimum size
    if len(cluster_points) < min_cluster_size:
        continue

    # Calculate the centroid of the cluster
    centroid = cluster_points.mean(axis=0)

    # Append cluster info (type and centroid) to the list
    cluster_type = "Window"
    cluster_info.append({"Cluster ID": cluster_id, "Cluster Type": cluster_type, "Centroid": centroid.tolist()})

    # Create a new point cloud for the cluster
    cluster_pcd = o3d.geometry.PointCloud()
    cluster_pcd.points = o3d.utility.Vector3dVector(cluster_points)

    # Save the cluster to a file
    output_file = f"window_{cluster_id}.ply"
    o3d.io.write_point_cloud(output_file, cluster_pcd)
    print(f"Cluster {cluster_id} saved as {output_file}")

# # Save the cluster info to a file or process it further
# print("Cluster Information:")
# for info in cluster_info:
#     print(f"Cluster Type: {info['Cluster Type']}, Centroid: {info['Centroid']}")

# Visualize the clustered point cloud
# Assign random colors to clusters for visualization
max_label = labels.max()
colors = np.random.uniform(0, 1, size=(max_label + 100, 3))  # Random colors
colors = np.vstack([colors, [0, 0, 0]])  # Add black for noise
pcd.colors = o3d.utility.Vector3dVector(colors[labels + 1])  # Map labels to colors



# Visualize the point cloud with clusters
o3d.visualization.draw_geometries([pcd], window_name="DBSCAN Clusters")


# # Save the centroids to a text file
# centroid_file = "cluster_centroids.txt"
# with open(centroid_file, "w") as f:
#     f.write("Cluster ID, Cluster Type, X, Y, Z\n")  # Header
#     for info in cluster_info:
#         cluster_id = info["Cluster ID"]
#         Cluster Type = info['Cluster Type']
#         centroid = info["Centroid"]
#         f.write(f"{cluster_id}, {centroid[0]:.6f}, {centroid[1]:.6f}, {centroid[2]:.6f}")

# print(f"Centroid coordinates saved to {centroid_file}")

# Save the cluster info to the console
print("Cluster Information:")
for info in cluster_info:
    print(f"Cluster_ID: {info['Cluster ID']}, Type: {info['Cluster Type']}, Centroid: {info['Centroid']}")

# Save the cluster information to a text file
cluster_info_file = "cluster_information.txt"
with open(cluster_info_file, "w") as f:
    # Write the header
    f.write("Cluster_ID, Type, Centroid_X, Centroid_Y, Centroid_Z\n")
    
    # Write each cluster's information
    for info in cluster_info:
        cluster_id = info["Cluster ID"]
        cluster_type = info["Cluster Type"]
        centroid = info["Centroid"]
        f.write(f"{cluster_id}, {cluster_type}, {centroid[0]:.6f}, {centroid[1]:.6f}, {centroid[2]:.6f}\n")

print(f"Cluster information saved to {cluster_info_file}")

# # Assign random colors to clusters for visualization
# max_label = labels.max()
# colors = np.random.uniform(0, 1, size=(max_label + 2, 3))  # Ensure sufficient colors
# colors[-1] = [0, 0, 0]  # Black for noise
# pcd.colors = o3d.utility.Vector3dVector(colors[labels + 1])  # Map labels to colors

# # Visualize the point cloud with clusters
# o3d.visualization.draw_geometries([pcd], window_name="DBSCAN Clusters")

eps=0.1, min_samples=5, clusters=0
eps=0.1, min_samples=10, clusters=0
eps=0.5, min_samples=5, clusters=85
eps=0.5, min_samples=10, clusters=85
Cluster 0 saved as window_0.ply
Cluster 1 saved as window_1.ply
Cluster 2 saved as window_2.ply
Cluster 3 saved as window_3.ply
Cluster 4 saved as window_4.ply
Cluster 5 saved as window_5.ply
Cluster 6 saved as window_6.ply
Cluster 7 saved as window_7.ply
Cluster 8 saved as window_8.ply
Cluster 9 saved as window_9.ply
Cluster 10 saved as window_10.ply
Cluster 11 saved as window_11.ply
Cluster 12 saved as window_12.ply
Cluster 13 saved as window_13.ply
Cluster 14 saved as window_14.ply
Cluster 15 saved as window_15.ply
Cluster 16 saved as window_16.ply
Cluster 17 saved as window_17.ply
Cluster 18 saved as window_18.ply
Cluster 19 saved as window_19.ply
Cluster 20 saved as window_20.ply
Cluster 21 saved as window_21.ply
Cluster 22 saved as window_22.ply
Cluster 23 saved as window_23.ply
Cluster 24 saved as window_24.ply
Cluster 25 saved as window

In [9]:
# use RANSAC to extract planes (walls) from ply file - RPR

import open3d as o3d
import numpy as np
import os

def multi_plane_segmentation(cloud_path):
    # Load your point cloud
    cloud = o3d.io.read_point_cloud(cloud_path)

    # Parameters
    distance_threshold = 0.1
    num_iterations = 1000
    plane_count = 0
    max_plane_count = 18


    # Create the output directory if it doesn't exist
    output_dir = "plane_all_RPR"
    os.makedirs(output_dir, exist_ok=True)
    normal_filepath = os.path.join(output_dir, "normal.txt")

    # Clean the contents of the output normal file
    with open(normal_filepath, 'w') as normal_file:
        normal_file.close()

    all_planes = o3d.geometry.PointCloud()

    # List to store detected planes
    detected_planes = []  # Initialized here to track all planes

    while True:
        # Run RANSAC to detect a plane
        plane_model, inliers = cloud.segment_plane(distance_threshold=distance_threshold,
                                                   ransac_n=3,
                                                   num_iterations=num_iterations)

        # Check if any planes were detected
        if len(inliers) == 0 or plane_count >= max_plane_count:
            break



        # Extract inliers
        inlier_cloud = cloud.select_by_index(inliers)

        # Save the plane cloud to a file
        plane_filename = f"plane_{plane_count:02}.ply"
        o3d.io.write_point_cloud(os.path.join(output_dir, plane_filename), inlier_cloud)

        # Write the plane's normal vector and offset to the file
        with open(normal_filepath, 'a') as outfile:
            normal = plane_model[:3]
            d = plane_model[3]
            outfile.write(f"Plane_ID:{plane_count};{normal[0]};{normal[1]};{normal[2]};{d}\n")

        # Color the inlier cloud
        colors = np.random.uniform(0, 1, size=(3,))
        inlier_cloud.paint_uniform_color(colors)

        # Add the plane cloud to the output cloud
        all_planes += inlier_cloud

        # Remove the inliers from the input cloud for the next iteration
        cloud = cloud.select_by_index(inliers, invert=True)

        # Store plane information
        detected_planes.append({
            "Plane_ID": plane_count,
            "Coefficients": plane_model.tolist(),  # A, B, C, D coefficients of the plane
            "Num_Points": len(inliers)
        })

        # Increment the plane count
        plane_count += 1

    # Visualize the result
    o3d.visualization.draw_geometries([all_planes])

    # Return detected planes
    return detected_planes

    # # Save detected plane information to a text file
    # plane_info_file = "detected_planes.txt"
    # with open(plane_info_file, "w") as f:
    #     f.write("Plane_ID, A, B, C, D, Num_Points\n")  # Header
    #     for plane in detected_planes:
    #         plane_id = plane["Plane_ID"]
    #         coefficients = plane["Coefficients"]
    #         num_points = plane["Num_Points"]
    #         f.write(f"{plane_id}, {coefficients[0]:.6f}, {coefficients[1]:.6f}, {coefficients[2]:.6f}, {coefficients[3]:.6f}, {num_points}\n")
    
    # print(f"Detected plane information saved to {plane_info_file}")


# Example usage
cloud_path = "..."
detected_planes = multi_plane_segmentation(cloud_path)


In [ ]:
# test for only extracting the vertical planes

import open3d as o3d
import numpy as np
import os

def multi_plane_segmentation(cloud_path):
    # Load your point cloud
    cloud = o3d.io.read_point_cloud(cloud_path)

    # Parameters
    distance_threshold = 0.1
    num_iterations = 1000
    plane_count = 0
    max_plane_count = 18

    # Create the output directory if it doesn't exist
    output_dir = "plane_all_RPR"
    os.makedirs(output_dir, exist_ok=True)
    normal_filepath = os.path.join(output_dir, "normal.txt")

    # Clean the contents of the output normal file
    with open(normal_filepath, 'w') as normal_file:
        normal_file.close()

    all_planes = o3d.geometry.PointCloud()

    # List to store detected planes
    detected_planes = []  # Initialized here to track all planes

    while True:
        # Run RANSAC to detect a plane
        plane_model, inliers = cloud.segment_plane(distance_threshold=distance_threshold,
                                                   ransac_n=3,
                                                   num_iterations=num_iterations)

        # Check if any planes were detected
        if len(inliers) == 0 or plane_count >= max_plane_count:
            break

        # Extract the normal vector from the plane model
        normal = np.array(plane_model[:3])

        # Adjust the plane to make it vertical
        normal[2] = 0  # Force the Z-component to be 0 to make the plane vertical
        normal = normal / np.linalg.norm(normal)  # Re-normalize the adjusted normal vector
        plane_model[:3] = normal.tolist()

        # Extract inliers
        inlier_cloud = cloud.select_by_index(inliers)

        # Save the plane cloud to a file
        plane_filename = f"plane_{plane_count:02}.ply"
        o3d.io.write_point_cloud(os.path.join(output_dir, plane_filename), inlier_cloud)

        # Write the plane's normal vector and offset to the file
        with open(normal_filepath, 'a') as outfile:
            d = plane_model[3]
            outfile.write(f"Plane_ID:{plane_count};{normal[0]};{normal[1]};{normal[2]};{d}\n")

        # Color the inlier cloud
        colors = np.random.uniform(0, 1, size=(3,))
        inlier_cloud.paint_uniform_color(colors)

        # Add the plane cloud to the output cloud
        all_planes += inlier_cloud

        # Remove the inliers from the input cloud for the next iteration
        cloud = cloud.select_by_index(inliers, invert=True)

        # Store plane information
        detected_planes.append({
            "Plane_ID": plane_count,
            "Coefficients": plane_model.tolist(),  # A, B, C, D coefficients of the plane
            "Num_Points": len(inliers)
        })

        # Increment the plane count
        plane_count += 1

    # Visualize the result
    o3d.visualization.draw_geometries([all_planes])

    # Return detected planes
    return detected_planes

def verify_vertical_planes(normal_filepath):
    """
    Verify if the extracted planes are vertical by checking their normal vectors.

    Parameters:
        normal_filepath (str): Path to the file containing normal vectors and plane data.

    Returns:
        None
    """
    with open(normal_filepath, 'r') as file:
        lines = file.readlines()

    print("Verification of vertical planes:")
    for line in lines:
        parts = line.strip().split(';')
        if len(parts) >= 5:
            plane_id = parts[0].split(':')[1]
            normal = np.array(list(map(float, parts[1:4])))

            # Calculate the deviation of the normal vector from the vertical plane
            z_component = normal[2]
            if abs(z_component) < 1e-6:  # Threshold for near-zero Z-component
                print(f"Plane {plane_id}: Verified as vertical.")
            else:
                print(f"Plane {plane_id}: Not vertical. Z-component = {z_component:.6f}")

# Example usage
cloud_path = "..."
detected_planes = multi_plane_segmentation(cloud_path)

# Verify the verticality of extracted planes
normal_filepath = "plane_all_RPR_test_vertical/normal.txt"
verify_vertical_planes(normal_filepath)


In [8]:
# use DBSCAN to extract each individual wall from planes (test for one plane)

import open3d as o3d
import numpy as np
from sklearn.cluster import DBSCAN

# Load the input point cloud
input_file = "plane_all_RPR/plane_17.ply"  # Update with your file name
pcd = o3d.io.read_point_cloud(input_file)

# Convert point cloud to NumPy array
points = np.asarray(pcd.points)

# Prepare the list to store cluster type and central point
cluster_info = []



# Initial parameters
eps_values = [0.1,0.5]  # Example range of eps
min_samples_values = [5,10]  # Example range of min_samples
min_cluster_size = 50  # Set the minimum number of points in a cluster


for eps in eps_values:
    for min_samples in min_samples_values:
        dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        labels = dbscan.fit_predict(points)
        num_clusters = len(set(labels)) - (1 if -1 in labels else 0)  # Exclude noise
        print(f"eps={eps}, min_samples={min_samples}, clusters={num_clusters}")



# Perform DBSCAN clustering
dbscan = DBSCAN(eps=eps, min_samples=min_samples)
labels = dbscan.fit_predict(points)

# Get unique cluster labels (excluding noise, which is labeled as -1)
unique_labels = set(labels)
unique_labels.discard(-1)  # Remove noise label if present

# Iterate over each cluster and save them as individual files
for cluster_id in unique_labels:
    # Extract points belonging to the current cluster
    cluster_points = points[labels == cluster_id]

    # Skip clusters smaller than the minimum size
    if len(cluster_points) < min_cluster_size:
        continue

    # Calculate the centroid of the cluster
    centroid = cluster_points.mean(axis=0)

    # Append cluster info (type and centroid) to the list
    cluster_type = "Wall"
    cluster_info.append({"Cluster ID": cluster_id, "Cluster Type": cluster_type, "Centroid": centroid.tolist()})

    # Create a new point cloud for the cluster
    cluster_pcd = o3d.geometry.PointCloud()
    cluster_pcd.points = o3d.utility.Vector3dVector(cluster_points)

    # Save the cluster to a file
    output_file = f"wall_{cluster_id}.ply"
    o3d.io.write_point_cloud(output_file, cluster_pcd)
    print(f"Cluster {cluster_id} saved as {output_file}")

# # Save the cluster info to a file or process it further
# print("Cluster Information:")
# for info in cluster_info:
#     print(f"Cluster Type: {info['Cluster Type']}, Centroid: {info['Centroid']}")

# Visualize the clustered point cloud
# Assign random colors to clusters for visualization
max_label = labels.max()
colors = np.random.uniform(0, 1, size=(max_label + 100, 3))  # Random colors
colors = np.vstack([colors, [0, 0, 0]])  # Add black for noise
pcd.colors = o3d.utility.Vector3dVector(colors[labels + 1])  # Map labels to colors



# Visualize the point cloud with clusters
o3d.visualization.draw_geometries([pcd], window_name="DBSCAN Clusters")



# Save the cluster info to the console
print("Cluster Information:")
for info in cluster_info:
    print(f"Cluster_ID: {info['Cluster ID']}, Type: {info['Cluster Type']}, Centroid: {info['Centroid']}")

# Save the cluster information to a text file
cluster_info_file = "cluster_information.txt"
with open(cluster_info_file, "w") as f:
    # Write the header
    f.write("Cluster_ID, Type, Centroid_X, Centroid_Y, Centroid_Z\n")
    
    # Write each cluster's information
    for info in cluster_info:
        cluster_id = info["Cluster ID"]
        cluster_type = info["Cluster Type"]
        centroid = info["Centroid"]
        f.write(f"{cluster_id}, {cluster_type}, {centroid[0]:.6f}, {centroid[1]:.6f}, {centroid[2]:.6f}\n")

print(f"Cluster information saved to {cluster_info_file}")



eps=0.1, min_samples=5, clusters=0
eps=0.1, min_samples=10, clusters=0
eps=0.5, min_samples=5, clusters=2
eps=0.5, min_samples=10, clusters=2
Cluster 0 saved as window_0.ply
Cluster 1 saved as window_1.ply
Cluster Information:
Cluster_ID: 0, Type: Wall, Centroid: [506402.39029731794, 311536.4282462463, 16.156553506858714]
Cluster_ID: 1, Type: Wall, Centroid: [506402.14883664, 311536.7602895327, 6.69457210965435]
Cluster information saved to cluster_information.txt


In [9]:
# use DBSCAN to extract each individual wall from planes


import open3d as o3d
import numpy as np
from sklearn.cluster import DBSCAN
import os

def process_ply_file(input_file, output_dir, eps=0.3, min_samples=5, min_cluster_size=50):
    """
    Process a single .ply file to extract clusters using DBSCAN.
    Save the extracted clusters as individual .ply files.

    Parameters:
        input_file (str): Path to the input .ply file.
        output_dir (str): Path to the directory where clusters will be saved.
        eps (float): DBSCAN epsilon parameter.
        min_samples (int): DBSCAN min_samples parameter.
        min_cluster_size (int): Minimum points in a cluster to be saved.
    """
    # Load the input point cloud
    pcd = o3d.io.read_point_cloud(input_file)
    points = np.asarray(pcd.points)

    # Perform DBSCAN clustering
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    labels = dbscan.fit_predict(points)

    # Get unique cluster labels (excluding noise, labeled as -1)
    unique_labels = set(labels)
    unique_labels.discard(-1)

    # Extract the base filename from the input file
    base_name = os.path.splitext(os.path.basename(input_file))[0]

    # Iterate over each cluster and save them as individual files
    cluster_count = 1
    for cluster_id in unique_labels:
        cluster_points = points[labels == cluster_id]

        # Skip clusters smaller than the minimum size
        if len(cluster_points) < min_cluster_size:
            continue

        # Create a new point cloud for the cluster
        cluster_pcd = o3d.geometry.PointCloud()
        cluster_pcd.points = o3d.utility.Vector3dVector(cluster_points)

        # Save the cluster to a file
        cluster_filename = f"{base_name}_{cluster_count:02}.ply"
        cluster_filepath = os.path.join(output_dir, cluster_filename)
        o3d.io.write_point_cloud(cluster_filepath, cluster_pcd)
        print(f"Cluster {cluster_count} from {input_file} saved to {cluster_filepath}")
        cluster_count += 1


def batch_process_ply_files(input_folder, output_folder, eps=0.3, min_samples=5, min_cluster_size=50):
    """
    Batch process all .ply files in a folder using DBSCAN.

    Parameters:
        input_folder (str): Path to the folder containing .ply files.
        output_folder (str): Path to the folder where clusters will be saved.
        eps (float): DBSCAN epsilon parameter.
        min_samples (int): DBSCAN min_samples parameter.
        min_cluster_size (int): Minimum points in a cluster to be saved.
    """
    # Ensure output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Process each .ply file in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith(".ply"):
            input_file = os.path.join(input_folder, filename)
            print(f"Processing {input_file}...")
            process_ply_file(input_file, output_folder, eps, min_samples, min_cluster_size)

# Example usage
input_folder = "plane_all_RPR"  # Replace with your input folder path
output_folder = "instance_wall_vertical_01"  # Replace with your output folder path
batch_process_ply_files(input_folder, output_folder, eps=0.3, min_samples=5, min_cluster_size=50)


Processing plane_all_RPR_test_vertical_01\plane_00.ply...
Cluster 1 from plane_all_RPR_test_vertical_01\plane_00.ply saved to instance_wall_vertical_01\plane_00_01.ply
Cluster 2 from plane_all_RPR_test_vertical_01\plane_00.ply saved to instance_wall_vertical_01\plane_00_02.ply
Cluster 3 from plane_all_RPR_test_vertical_01\plane_00.ply saved to instance_wall_vertical_01\plane_00_03.ply
Processing plane_all_RPR_test_vertical_01\plane_01.ply...
Cluster 1 from plane_all_RPR_test_vertical_01\plane_01.ply saved to instance_wall_vertical_01\plane_01_01.ply
Cluster 2 from plane_all_RPR_test_vertical_01\plane_01.ply saved to instance_wall_vertical_01\plane_01_02.ply
Cluster 3 from plane_all_RPR_test_vertical_01\plane_01.ply saved to instance_wall_vertical_01\plane_01_03.ply
Cluster 4 from plane_all_RPR_test_vertical_01\plane_01.ply saved to instance_wall_vertical_01\plane_01_04.ply
Processing plane_all_RPR_test_vertical_01\plane_02.ply...
Cluster 1 from plane_all_RPR_test_vertical_01\plane_02.p

In [10]:
# use DBSCAN to extract each individual wall (vertical_02)


import open3d as o3d
import numpy as np
from sklearn.cluster import DBSCAN
import os

def process_ply_file(input_file, output_dir, eps=0.3, min_samples=5, min_cluster_size=50):
    """
    Process a single .ply file to extract clusters using DBSCAN.
    Save the extracted clusters as individual .ply files.

    Parameters:
        input_file (str): Path to the input .ply file.
        output_dir (str): Path to the directory where clusters will be saved.
        eps (float): DBSCAN epsilon parameter.
        min_samples (int): DBSCAN min_samples parameter.
        min_cluster_size (int): Minimum points in a cluster to be saved.
    """
    # Load the input point cloud
    pcd = o3d.io.read_point_cloud(input_file)
    points = np.asarray(pcd.points)

    # Perform DBSCAN clustering
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    labels = dbscan.fit_predict(points)

    # Get unique cluster labels (excluding noise, labeled as -1)
    unique_labels = set(labels)
    unique_labels.discard(-1)

    # Extract the base filename from the input file
    base_name = os.path.splitext(os.path.basename(input_file))[0]

    # Iterate over each cluster and save them as individual files
    cluster_count = 1
    for cluster_id in unique_labels:
        cluster_points = points[labels == cluster_id]

        # Skip clusters smaller than the minimum size
        if len(cluster_points) < min_cluster_size:
            continue

        # Create a new point cloud for the cluster
        cluster_pcd = o3d.geometry.PointCloud()
        cluster_pcd.points = o3d.utility.Vector3dVector(cluster_points)

        # Save the cluster to a file
        cluster_filename = f"{base_name}_{cluster_count:02}.ply"
        cluster_filepath = os.path.join(output_dir, cluster_filename)
        o3d.io.write_point_cloud(cluster_filepath, cluster_pcd)
        print(f"Cluster {cluster_count} from {input_file} saved to {cluster_filepath}")
        cluster_count += 1


def batch_process_ply_files(input_folder, output_folder, eps=0.3, min_samples=5, min_cluster_size=50):
    """
    Batch process all .ply files in a folder using DBSCAN.

    Parameters:
        input_folder (str): Path to the folder containing .ply files.
        output_folder (str): Path to the folder where clusters will be saved.
        eps (float): DBSCAN epsilon parameter.
        min_samples (int): DBSCAN min_samples parameter.
        min_cluster_size (int): Minimum points in a cluster to be saved.
    """
    # Ensure output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Process each .ply file in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith(".ply"):
            input_file = os.path.join(input_folder, filename)
            print(f"Processing {input_file}...")
            process_ply_file(input_file, output_folder, eps, min_samples, min_cluster_size)

# Example usage
input_folder = "plane_all_RPR_test_vertical_02"  
output_folder = "instance_wall_vertical_02"  
batch_process_ply_files(input_folder, output_folder, eps=0.3, min_samples=5, min_cluster_size=50)


Processing plane_all_RPR_test_vertical_02\all_planes.ply...
Cluster 1 from plane_all_RPR_test_vertical_02\all_planes.ply saved to instance_wall_vertical_02\all_planes_01.ply
Cluster 2 from plane_all_RPR_test_vertical_02\all_planes.ply saved to instance_wall_vertical_02\all_planes_02.ply
Cluster 3 from plane_all_RPR_test_vertical_02\all_planes.ply saved to instance_wall_vertical_02\all_planes_03.ply
Cluster 4 from plane_all_RPR_test_vertical_02\all_planes.ply saved to instance_wall_vertical_02\all_planes_04.ply
Cluster 5 from plane_all_RPR_test_vertical_02\all_planes.ply saved to instance_wall_vertical_02\all_planes_05.ply
Cluster 6 from plane_all_RPR_test_vertical_02\all_planes.ply saved to instance_wall_vertical_02\all_planes_06.ply
Cluster 7 from plane_all_RPR_test_vertical_02\all_planes.ply saved to instance_wall_vertical_02\all_planes_07.ply
Cluster 8 from plane_all_RPR_test_vertical_02\all_planes.ply saved to instance_wall_vertical_02\all_planes_08.ply
Cluster 9 from plane_all_RPR

In [11]:
# extract the mapping table to rename the wall instance_vertical_02
import os
import shutil

def rename_instance_walls(input_folder, output_folder):
    """
    Rename instance wall files in sequential order and generate a mapping table.

    Parameters:
        input_folder (str): Path to the folder containing `plane_xx_xx` files.
        output_folder (str): Path to save renamed wall files.
    
    Returns:
        list of dict: Mapping table with source and renamed file information.
    """
    # Ensure output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Initialize counters and mapping list
    wall_counter = 0
    mapping_table = []

    # Process all files in the input folder
    for file_name in sorted(os.listdir(input_folder)):
        if file_name.endswith(".ply") and file_name.startswith("plane_"):
            # Define the new name for the wall
            new_name = f"plane_{str(wall_counter).zfill(3)}.ply"
            input_path = os.path.join(input_folder, file_name)
            output_path = os.path.join(output_folder, new_name)

            # Rename the file by copying it to the new location
            shutil.copy(input_path, output_path)

            # Add to mapping table
            mapping_table.append({
                "source": file_name,
                "wall_instance": new_name
            })

            # Increment the wall counter
            wall_counter += 1

    # Return the mapping table
    return mapping_table

def save_mapping_table(mapping_table, mapping_file):
    """
    Save the mapping table to a text file.

    Parameters:
        mapping_table (list of dict): Mapping table with source and renamed file information.
        mapping_file (str): Path to save the mapping table.
    """
    with open(mapping_file, "w") as f:
        # Write the header
        f.write("Source, Wall_Instance\n")
        # Write each mapping
        for entry in mapping_table:
            f.write(f"{entry['source']} -> {entry['wall_instance']}\n")
    print(f"Mapping table saved to {mapping_file}")

# Example usage
input_folder = "instance_wall_vertical_02"  # Replace with the folder containing `plane_xx_xx.ply` files
output_folder = "renamed_walls_vertical_02"  # Folder for renamed wall files
mapping_file = "renamed_walls_vertical_02/mapping_table.txt"  # File to save the mapping table

# Rename files and generate mapping table
mapping_table = rename_instance_walls(input_folder, output_folder)

# Save the mapping table
save_mapping_table(mapping_table, mapping_file)


Mapping table saved to renamed_walls_vertical_02/mapping_table.txt


In [12]:
# extract the plane A B C D information from the normal.txt and mapped into the new naming wall instance_vertical_02


import os

def load_plane_coefficients(normal_file):
    """
    Load plane coefficients (A, B, C, D) from the normal.txt file.

    Parameters:
        normal_file (str): Path to the normal.txt file.

    Returns:
        dict: Dictionary mapping plane IDs (e.g., plane_00) to coefficients [A, B, C, D].
    """
    plane_coefficients = {}
    with open(normal_file, "r") as f:
        for line in f:
            parts = line.strip().split(";")
            if len(parts) >= 5:  # Ensure the line has enough fields
                plane_id = f"plane_{parts[0].split(':')[1].zfill(2)}"  # Extract and zero-pad plane ID
                coefficients = list(map(float, parts[1:5]))  # Extract A, B, C, D
                plane_coefficients[plane_id] = coefficients
    return plane_coefficients


def map_plane_coefficients_to_walls(mapping_file, plane_coefficients, output_file):
    """
    Map plane coefficients to renamed wall instances based on the mapping table.

    Parameters:
        mapping_file (str): Path to the mapping table file.
        plane_coefficients (dict): Dictionary of plane coefficients.
        output_file (str): Path to save the mapping of wall instances to plane coefficients.
    """
    with open(mapping_file, "r") as f:
        lines = f.readlines()

    # Skip the header
    lines = lines[1:]

    # Map walls to plane coefficients
    wall_plane_mapping = []
    for line in lines:
        source, wall_instance = line.strip().split(" -> ")
        plane_id = f"plane_{source.split('_')[1]}"  # Correctly extract plane ID, e.g., plane_01
        if plane_id in plane_coefficients:
            coefficients = plane_coefficients[plane_id]
            wall_plane_mapping.append({
                "wall_instance": wall_instance,
                "coefficients": coefficients
            })

    # Save the mapping to a text file
    with open(output_file, "w") as f:
        f.write("Wall_Instance, A, B, C, D\n")
        for entry in wall_plane_mapping:
            coeffs = entry["coefficients"]
            f.write(f"{entry['wall_instance']}, {coeffs[0]}, {coeffs[1]}, {coeffs[2]}, {coeffs[3]}\n")

    print(f"Wall-to-plane coefficient mapping saved to {output_file}")

# Example usage
normal_file = "plane_all_RPR_test_vertical_02/normal.txt"  # Path to normal.txt
mapping_file = "renamed_walls_vertical_02/mapping_table.txt"  # Path to the mapping table
output_file = "renamed_walls_vertical_02/wall_plane_coefficients.txt"  # File to save the wall-to-plane mapping

# Load coefficients from normal.txt
plane_coefficients = load_plane_coefficients(normal_file)
print("Loaded plane coefficients:")
for plane_id, coeffs in plane_coefficients.items():
    print(f"{plane_id}: A={coeffs[0]}, B={coeffs[1]}, C={coeffs[2]}, D={coeffs[3]}")

# Map plane coefficients to walls
map_plane_coefficients_to_walls(mapping_file, plane_coefficients, output_file)


Loaded plane coefficients:
plane_00: A=-0.669837, B=0.742508, C=0.000475, D=107908.549886
plane_01: A=0.775761, B=0.630906, C=0.012341, D=-589342.199751
plane_02: A=0.774995, B=0.631853, C=0.01203, D=-589264.867506
plane_03: A=-0.671126, B=0.741339, C=0.002439, D=108899.397653
plane_04: A=-0.669423, B=0.742881, C=0.000796, D=107566.112885
plane_05: A=-0.669781, B=0.742559, C=0.000151, D=107855.412592
plane_06: A=0.773415, B=0.633637, C=0.01826, D=-589004.721031
plane_07: A=0.779642, B=0.625974, C=0.017716, D=-589782.894861
plane_08: A=-0.674168, B=0.738574, C=-0.002307, D=111329.090831
plane_09: A=0.742768, B=0.669544, C=0.002527, D=-584772.138451
plane_10: A=0.73635, B=0.676586, C=0.004475, D=-583683.256907
plane_11: A=-0.669037, B=0.743226, C=0.001975, D=107265.527198
plane_12: A=0.778359, B=0.627581, C=0.017315, D=-589636.103093
plane_13: A=-0.667181, B=0.744895, C=-0.000564, D=105794.147187
plane_14: A=-0.673149, B=0.739506, C=-0.000806, D=110511.798297
plane_15: A=-0.636061, B=0.7

In [35]:
# find the convex_hull (not used)

import open3d as o3d
import numpy as np
from scipy.spatial import ConvexHull
import os

def export_convex_hull(input_file, output_file):
    """
    Compute and export the convex hull of a point cloud.

    Parameters:
        input_file (str): Path to the input .ply file.
        output_file (str): Path to save the convex hull .ply file.
    """
    # Load the input point cloud
    pcd = o3d.io.read_point_cloud(input_file)
    points = np.asarray(pcd.points)

    # Ensure there are enough points for a convex hull
    if len(points) < 4:
        print("Not enough points to compute a convex hull.")
        return

    # Compute the convex hull
    hull = ConvexHull(points)

    # Extract the vertices of the hull
    hull_vertices = points[hull.vertices]

    # Create a new point cloud for the convex hull
    hull_pcd = o3d.geometry.PointCloud()
    hull_pcd.points = o3d.utility.Vector3dVector(hull_vertices)

    # Save the convex hull as a .ply file
    o3d.io.write_point_cloud(output_file, hull_pcd)
    print(f"Convex hull saved to {output_file}")

# Example usage
input_file = "instance_wall/plane_12_02.ply"  # Replace with your input .ply file
output_file = "ply_12_02_convex_hull.ply"  # Replace with your desired output file
export_convex_hull(input_file, output_file)


Convex hull saved to ply_12_02_convex_hull.ply


In [34]:
# not used

import open3d as o3d
import numpy as np

def convert_ply_to_xyz(input_ply_file, output_xyz_file):
    """
    Convert a .ply file into an .xyz file.

    Parameters:
        input_ply_file (str): Path to the input .ply file.
        output_xyz_file (str): Path to save the output .xyz file.
    """
    # Load the point cloud from the .ply file
    pcd = o3d.io.read_point_cloud(input_ply_file)
    points = np.asarray(pcd.points)

    # Save the points to an .xyz file
    np.savetxt(output_xyz_file, points, fmt="%.6f", delimiter=" ")
    print(f"Converted {input_ply_file} to {output_xyz_file}")

# Example usage
input_ply_file = "ply_12_02_convex_hull.ply"  # Replace with your input .ply file
output_xyz_file = "ply_12_02_convex_hull.xyz"  # Replace with your desired output .xyz file
convert_ply_to_xyz(input_ply_file, output_xyz_file)

[Open3D WARNING] Read PLY failed: unable to open file: ply_12_02_convex_hull.ply
Converted ply_12_02_convex_hull.ply to ply_12_02_convex_hull.xyz


In [ ]:
# not used

import open3d as o3d
import numpy as np

def convert_ply_to_xyz(input_ply_file, output_xyz_file):
    """
    Convert a .ply file into an .xyz file.

    Parameters:
        input_ply_file (str): Path to the input .ply file.
        output_xyz_file (str): Path to save the output .xyz file.
    """
    # Load the point cloud from the .ply file
    pcd = o3d.io.read_point_cloud(input_ply_file)
    points = np.asarray(pcd.points)

    # Save the points to an .xyz file
    np.savetxt(output_xyz_file, points, fmt="%.6f", delimiter=" ")
    print(f"Converted {input_ply_file} to {output_xyz_file}")

# Example usage
input_ply_file = "ply_12_02_convex_hull.ply"  # Replace with your input .ply file
output_xyz_file = "ply_12_02_convex_hull.xyz"  # Replace with your desired output .xyz file
convert_ply_to_xyz(input_ply_file, output_xyz_file)

In [38]:
# best fitted 2D plane with the boundary of each plane (also need the information of parameter of the each plane, which are A, B, C, D) (test for individual vertical ply)

import open3d as o3d
import numpy as np

def generate_boundary_rectangle_on_plane(input_ply_file, plane_params, output_mesh_file):
    """
    Generate a boundary rectangle on a given 3D plane for points in a .ply file.

    Parameters:
        input_ply_file (str): Path to the input .ply file.
        plane_params (list or np.ndarray): Plane equation coefficients [a, b, c, d].
        output_mesh_file (str): Path to save the rectangular plane as a .ply file.
    """
    # Load the point cloud
    pcd = o3d.io.read_point_cloud(input_ply_file)
    points = np.asarray(pcd.points)

    # Extract the normal vector and offset from the plane equation
    plane_normal = np.array(plane_params[:3])  # [a, b, c]
    plane_offset = plane_params[3]  # d

    # Normalize the plane normal
    normal = plane_normal / np.linalg.norm(plane_normal)

    # Project points to the plane
    distances = points @ normal + plane_offset  # Signed distances to the plane
    projected_points = points - np.outer(distances, normal)  # Projected points

    # Find two orthogonal vectors in the plane
    u = np.cross([1, 0, 0], normal)  # Arbitrary vector orthogonal to the normal
    if np.linalg.norm(u) < 1e-6:  # Handle edge case where normal ~ [1, 0, 0]
        u = np.cross([0, 1, 0], normal)
    u = u / np.linalg.norm(u)
    v = np.cross(normal, u)

    # Transform points to the 2D plane coordinate system
    points_2d = np.dot(projected_points, np.array([u, v]).T)

    # Find the bounding rectangle in the 2D plane
    min_x, min_y = points_2d.min(axis=0)
    max_x, max_y = points_2d.max(axis=0)
    rectangle_2d = np.array([
        [min_x, min_y],
        [max_x, min_y],
        [max_x, max_y],
        [min_x, max_y]
    ])

    # Transform the rectangle back to 3D
    rectangle_3d = rectangle_2d @ np.array([u, v]) + normal * -plane_offset  # Map to 3D

    # Create a mesh for the rectangle
    mesh = o3d.geometry.TriangleMesh()
    mesh.vertices = o3d.utility.Vector3dVector(rectangle_3d)
    mesh.triangles = o3d.utility.Vector3iVector([[0, 1, 2], [2, 3, 0]])  # Two triangles for a rectangle
    mesh.compute_vertex_normals()

    # Save the rectangle as a .ply file
    o3d.io.write_triangle_mesh(output_mesh_file, mesh)
    print(f"Fitted boundary rectangle saved to {output_mesh_file}")

# Example usage
input_ply_file = "instance_wall/plane_12_02.ply"  # Replace with your input .ply file
plane_params = [-0.6688143169130036, 0.7434289125366979, 0.0009281684627058093, 107089.58475491752]  # Replace with your plane equation coefficients [a, b, c, d]



output_mesh_file = "plane_12_02_boundary_rectangle.ply"  # Replace with your desired output .ply file

generate_boundary_rectangle_on_plane(input_ply_file, plane_params, output_mesh_file)


Fitted boundary rectangle saved to plane_12_02_boundary_rectangle.ply


In [13]:
# best fitted 2D plane with the boundary of each plane (also need the information of parameter of the each plane, which are A, B, C, D) (test for all vertical ply_02_final)


import os
import open3d as o3d
import numpy as np


def load_plane_coefficients(coefficients_file):
    """
    Load plane coefficients from a file.

    Parameters:
        coefficients_file (str): Path to the wall_plane_coefficients.txt file.

    Returns:
        dict: Mapping of file names to plane coefficients.
    """
    plane_coefficients = {}
    with open(coefficients_file, "r") as f:
        lines = f.readlines()[1:]  # Skip the header
        for line in lines:
            parts = line.strip().split(",")
            file_name = parts[0].strip()
            coefficients = list(map(float, parts[1:5]))  # Extract A, B, C, D
            plane_coefficients[file_name] = coefficients
    return plane_coefficients


def generate_boundary_rectangle_on_plane(input_ply_file, plane_params, output_mesh_file):
    """
    Generate a boundary rectangle on a given 3D plane for points in a .ply file.

    Parameters:
        input_ply_file (str): Path to the input .ply file.
        plane_params (list or np.ndarray): Plane equation coefficients [a, b, c, d].
        output_mesh_file (str): Path to save the rectangular plane as a .ply file.
    """
    # Load the point cloud
    pcd = o3d.io.read_point_cloud(input_ply_file)
    points = np.asarray(pcd.points)

    # Extract the normal vector and offset from the plane equation
    plane_normal = np.array(plane_params[:3])  # [a, b, c]
    plane_offset = plane_params[3]  # d

    # Normalize the plane normal
    normal = plane_normal / np.linalg.norm(plane_normal)

    # Project points to the plane
    distances = points @ normal + plane_offset  # Signed distances to the plane
    projected_points = points - np.outer(distances, normal)  # Projected points

    # Find two orthogonal vectors in the plane
    u = np.cross([1, 0, 0], normal)  # Arbitrary vector orthogonal to the normal
    if np.linalg.norm(u) < 1e-6:  # Handle edge case where normal ~ [1, 0, 0]
        u = np.cross([0, 1, 0], normal)
    u = u / np.linalg.norm(u)
    v = np.cross(normal, u)

    # Transform points to the 2D plane coordinate system
    points_2d = np.dot(projected_points, np.array([u, v]).T)

    # Find the bounding rectangle in the 2D plane
    min_x, min_y = points_2d.min(axis=0)
    max_x, max_y = points_2d.max(axis=0)
    rectangle_2d = np.array([
        [min_x, min_y],
        [max_x, min_y],
        [max_x, max_y],
        [min_x, max_y]
    ])

    # Transform the rectangle back to 3D
    rectangle_3d = rectangle_2d @ np.array([u, v]) + normal * -plane_offset  # Map to 3D

    # Create a mesh for the rectangle
    mesh = o3d.geometry.TriangleMesh()
    mesh.vertices = o3d.utility.Vector3dVector(rectangle_3d)
    mesh.triangles = o3d.utility.Vector3iVector([[0, 1, 2], [2, 3, 0]])  # Two triangles for a rectangle
    mesh.compute_vertex_normals()

    # Save the rectangle as a .ply file
    o3d.io.write_triangle_mesh(output_mesh_file, mesh)
    print(f"Fitted boundary rectangle saved to {output_mesh_file}")


def process_all_ply_files(input_folder, coefficients_file, output_folder):
    """
    Process all .ply files in the input folder, generate boundary rectangles, and save them.

    Parameters:
        input_folder (str): Path to the folder containing .ply files.
        coefficients_file (str): Path to the wall_plane_coefficients.txt file.
        output_folder (str): Path to save the rectangular meshes.
    """
    # Load the plane coefficients
    plane_coefficients = load_plane_coefficients(coefficients_file)

    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Iterate through all .ply files in the input folder
    for file_name in sorted(os.listdir(input_folder)):
        if file_name.endswith(".ply"):
            input_ply_file = os.path.join(input_folder, file_name)

            # Retrieve the plane coefficients for this file
            if file_name in plane_coefficients:
                plane_params = plane_coefficients[file_name]

                # Define the output file name
                output_mesh_file = os.path.join(output_folder, f"{file_name.split('.')[0]}_boundary_rectangle.ply")

                # Generate the boundary rectangle
                generate_boundary_rectangle_on_plane(input_ply_file, plane_params, output_mesh_file)
            else:
                print(f"Plane coefficients not found for {file_name}")


# Example usage
input_folder = "renamed_walls_vertical_02"  # Folder containing renamed .ply files
coefficients_file = "renamed_walls_vertical_02/wall_plane_coefficients.txt"  # File with plane coefficients
output_folder = "boundary_rectangles_vertical_02"  # Folder to save output meshes

process_all_ply_files(input_folder, coefficients_file, output_folder)


Fitted boundary rectangle saved to boundary_rectangles_vertical_02\plane_000_boundary_rectangle.ply
Fitted boundary rectangle saved to boundary_rectangles_vertical_02\plane_001_boundary_rectangle.ply
Fitted boundary rectangle saved to boundary_rectangles_vertical_02\plane_002_boundary_rectangle.ply
Fitted boundary rectangle saved to boundary_rectangles_vertical_02\plane_003_boundary_rectangle.ply
Fitted boundary rectangle saved to boundary_rectangles_vertical_02\plane_004_boundary_rectangle.ply
Fitted boundary rectangle saved to boundary_rectangles_vertical_02\plane_005_boundary_rectangle.ply
Fitted boundary rectangle saved to boundary_rectangles_vertical_02\plane_006_boundary_rectangle.ply
Fitted boundary rectangle saved to boundary_rectangles_vertical_02\plane_007_boundary_rectangle.ply
Fitted boundary rectangle saved to boundary_rectangles_vertical_02\plane_008_boundary_rectangle.ply
Fitted boundary rectangle saved to boundary_rectangles_vertical_02\plane_009_boundary_rectangle.ply
